In [1]:
# GOAL: fit KNN, RF, and CATBOOST, including hyperparameter tuning
# and assess based on (r)mse and R^2.

In [38]:
# installs and imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import sklearn.metrics as skm

In [7]:
# import train and test data
Xtrain_filled = pd.read_csv("Xtrain_filled.csv",index_col="School Name")
ytrain = pd.read_csv("ytrain.csv",index_col="School Name").squeeze()
Xtest_filled = pd.read_csv("Xtest_filled.csv",index_col="School Name")
ytest0 = pd.read_csv("ytest.csv",index_col="School Name").squeeze()

## KNN Regression

In [9]:
knn0 = KNeighborsRegressor()
knn0.fit(Xtrain_filled,ytrain)

KNeighborsRegressor()

In [28]:
np.arange(4,20,1)

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [29]:
# set parms for hyperparameter tuning
parms = {
    "n_neighbors": np.arange(4,20,1),
    "weights": ["uniform","distance"]
}
knn = KNeighborsRegressor() # initialize knn
gc_knn = GridSearchCV(estimator=knn,param_grid=parms, cv = 10,
                      scoring = "neg_root_mean_squared_error")

In [30]:
# fit/run gcv for knn
gc_knn.fit(Xtrain_filled,ytrain)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                         'weights': ['uniform', 'distance']},
             scoring='neg_root_mean_squared_error')

In [46]:
print(gc_knn.best_params_)
print(-gc_knn.best_score_/np.std(ytrain))
best_knn = KNeighborsRegressor(n_neighbors= 12, weights= 'distance').fit(Xtrain_filled,ytrain)
best_knn

{'n_neighbors': 12, 'weights': 'distance'}
0.7358773686754505


KNeighborsRegressor(n_neighbors=12, weights='distance')

In [47]:
# metrics
print("Test R^2:", skm.r2_score(ytest0,best_knn.predict(Xtest_filled)))
print("Test RMSE:", np.sqrt(skm.mean_squared_error(ytest0,best_knn.predict(Xtest_filled))))
print("Test Scaled RMSE:", np.sqrt(skm.mean_squared_error(ytest0,best_knn.predict(Xtest_filled)))/np.std(ytest0))

Test R^2: 0.45508212239720847
Test RMSE: 8192.634811123551
Test Scaled RMSE: 0.7381855306105582


## Random Forest

In [48]:
rf = RandomForestRegressor(criterion="squared_error")
rf_parms = {

}

In [ ]:
gcv_rf = GridSearchCV(rf,param_grid=rf_parms)

## CATBOOST
A nice perk of cat boost: it looks like it may allow you to specify which predictors are categorical, and will do (one-hot) encoding for you
Can try it both ways and see if performance differs significantly.